In [1]:
%%capture
!pip install transformers>=4.40.1 accelerate>=0.27.2

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


In [3]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [4]:
model = AutoModelForCausalLM.from_pretrained(
  "microsoft/Phi-3-mini-4k-instruct",
  #device_map="cuda",
  torch_dtype="auto",
  trust_remote_code=True,
)

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [5]:
# Create a pipeline
generator = pipeline(
  "text-generation",
  model=model,
  tokenizer=tokenizer,
  return_full_text=False,
  max_new_tokens=50,
  do_sample=False,
)

Device set to use cpu


There’s a specific word used in machine learning to describe models that consume
their earlier predictions to make later predictions (e.g., the model’s first generated
token is used to generate the second token). They’re called autoregressive models.

That is why you’ll hear text generation LLMs being called autoregressive models. This
is often used to differentiate text generation models from text representation models
like BERT, which are not autoregressive.

In [6]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."
output = generator(prompt)
print(output[0]['generated_text'])

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


 Mention the steps you're taking to prevent it in the future.

Dear Sarah,

I hope this message finds you well. I am writing to express my deepest apologies for the unfortunate incident that occurred in


We saw in the previous chapter how tokenizers break down the text into a
sequence of token IDs that then become the input to the model.

The tokenizer is followed by the neural network: a stack of Transformer blocks that
do all of the processing. That stack is then followed by the LM head, which translates
the output of the stack into probability scores for what the most likely next token is.

The LM head is a simple neural network layer itself. It is one of multiple possible
“heads” to attach to a stack of Transformer blocks to build different kinds of systems.
Other kinds of Transformer heads include sequence classification heads and token
classification heads.

In [7]:
model

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out_features=3206

Low temperature (e.g., close to 0): The model behaves more like greedy decoding, favoring high-probability tokens and producing more deterministic outputs.

High temperature (e.g., >1): The model becomes more random, giving lower-probability tokens a higher chance of being selected, which can lead to more diverse but potentially less coherent outputs.

In [8]:
prompt = "The capital of France is"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids

tensor([[ 450, 7483,  310, 3444,  338]])

In [9]:
model_output = model.model(input_ids)

In [10]:
lm_head_output = model.lm_head(model_output[0])

In [11]:
lm_head_output

tensor([[[25.1250, 25.2500, 23.2500,  ..., 19.3750, 19.3750, 19.3750],
         [31.0000, 31.5000, 26.0000,  ..., 25.8750, 25.8750, 25.8750],
         [31.3750, 28.7500, 31.0000,  ..., 26.1250, 26.1250, 26.1250],
         [33.0000, 31.8750, 36.0000,  ..., 27.8750, 27.8750, 27.8750],
         [27.8750, 29.5000, 28.1250,  ..., 20.5000, 20.5000, 20.5000]]],
       dtype=torch.bfloat16, grad_fn=<UnsafeViewBackward0>)

In [13]:
token_id = lm_head_output[0,-1].argmax(-1)
token_id

tensor(3681)

In [14]:
tokenizer.decode(token_id)

'Paris'

Probability Distribution: The model outputs a probability distribution over the vocabulary for the next token.

Decoding Strategies:

Greedy Decoding: Always picks the highest-probability token. Simple but can lead to repetitive outputs.

Sampling: Randomly selects a token based on its probability, introducing diversity and creativity.

Temperature: Controls the randomness of sampling. Lower temperatures make the model more deterministic, while higher temperatures increase randomness.

Code Implementation: The provided code demonstrates how to tokenize an input, pass it through the model, and decode the output to generate text.

This process is fundamental to how large language models generate text, and the choice of decoding strategy plays a crucial role in the quality and style of the output.

In [15]:
model_output[0].shape

torch.Size([1, 5, 3072])

In [16]:
lm_head_output.shape

torch.Size([1, 5, 32064])

In [18]:
prompt = "Write a very long email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

In [19]:
# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

In [20]:
input_ids

tensor([[14350,   263,  1407,  1472,  4876, 27746,  5281,   304, 19235,   363,
           278, 25305,   293, 16423,   292,   286,   728,   481, 29889, 12027,
          7420,   920,   372,  9559, 29889]])

A Transformer block is made up of two successive components:
1. The attention layer is mainly concerned with incorporating relevant information
from other input tokens and positions
2. The feedforward layer houses the majority of the model’s processing capacity